# Implementation of the custom comparison tool developed for this work

### Read Data into Dict

In [1]:
from collections import defaultdict
import os
import socket
from IPython.display import HTML
import ipywidgets as widgets
import pickle

import sys
sys.path.append('../preprocessing')

from read_into_dicts import DocReader
from transformers import AutoTokenizer 


import matplotlib.pyplot as plt
import yaml

# pymupdf
import fitz
from random import random
import time
import requests
import io
# import pytesseract
from difflib import SequenceMatcher
from PIL import Image, ImageDraw, ImageFont
from difflib import SequenceMatcher
import multiprocessing
from transformers import logging as transformers_logging

2024-05-28 07:55:30.034147: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 07:55:33.118358: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
is_VM = "cpu" in socket.gethostname() or "gpu" in socket.gethostname() or "login" in socket.gethostname() # distinction made since code was sometimes run on local machine or on VM
cpu_count = multiprocessing.cpu_count()

with open('../Model/configs/config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)

MODEL = config["general"]["MODEL"]

tokenizer = AutoTokenizer.from_pretrained(MODEL)
print(f"{cpu_count} CPUs found on machine.")

class suppress_transformers_logging:
    def __enter__(self):
        # Suppress logging of warnings in the transformers library
        transformers_logging.set_verbosity_error()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        # Reset the logging level to its default (warning)
        transformers_logging.set_verbosity_warning()


128 CPUs found on machine.


### Read data

In [3]:
doc_reader = DocReader(model=MODEL, main_tokenizer=tokenizer)
doc_reader.extract_title=True
doc_reader.extract_doc_long_id=True
doc_reader.refine_regions=True
doc_reader.create_original_snippets=True
sample_file_path = "/home/tlh45/rds/hpc-work/28245V231219/17144355.xml"
# doc_reader.process_xml_file(sample_file_path)

In [4]:
# GENERAL:
PROJECT = "AML"
DATA_DICT_FILE_PATH = config["general"]['DATA_DICT_FILE_PATH'][PROJECT]
DATA_PATH = config["general"]["DATA_PATH"]

In [5]:
folder_path = '../28245V231219/' if not is_VM else '/home/tlh45/rds/hpc-work/28245V231219_AML'
data_dict_location = "../preprocessing/data_dicts/xx.pkl"  if not is_VM else f'{DATA_PATH}/{DATA_DICT_FILE_PATH}'
# data_dict_location = None

docReader = DocReader(MODEL, tokenizer)
if data_dict_location:
    with open(data_dict_location, 'rb') as handle:
        data_dict = pickle.load(handle)
    print(f"Data_dict loaded from {data_dict_location} with {len(data_dict)} entries.")
else:
    with suppress_transformers_logging():
        data_dict = docReader.preprocess_folder(preprocess=True, folder_path=folder_path, num_workers=min(20, cpu_count), data_size=1172, chunksize=1
                                                , extract_title=True, extract_doc_long_id=True, refine_regions=True, create_original_snippets=True, add_full_page=True
                                                , data_dict_folder=f"{DATA_PATH}/preprocessing/data_dicts/28245V231219_AML", file_name_additional_suffix="-10-04") # 21-02-new

Data_dict loaded from /home/tlh45/rds/hpc-work/preprocessing/data_dicts/28245V231219_AML/data_dict_roberta-base-10-04.pkl with 1149 entries.


In [6]:
if not data_dict_location:
    with open(f'{DATA_PATH}/preprocessing/data_dicts/28245V231219_AML/data_dict_{MODEL.replace("/", "-")}-10-04.pkl', 'wb') as handle:
        pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Analyse tags

Note that this is similar to what the detailed_labels_handler would handle, but we keep it here for a separated simplified version

In [7]:
from collections import defaultdict, Counter

def extract_and_analyze_tags(data_dict):
    # Stores unique tags for each level
    level1_tags = Counter()
    level2_tags = Counter()
    level3_tags = Counter()
    
    # Store unique combinations of full tags
    full_tags = Counter()
    
    # Nested function to process each tag
    def process_tag(tag):
        parts = tag.split('-')
        if len(parts) >= 1:
            level1_tags[parts[0]] += 1
        if len(parts) >= 2:
            level2_tags[parts[0] + '-' + parts[1]] += 1
        if len(parts) == 3:
            level3_tags['-'.join(parts)] += 1
        full_tags[tag] += 1

    # Iterate through data_dict to extract tags
    for doc_id, doc_data in data_dict.items():
        for page_index, page_data in doc_data.items():
            if 'refined_regions' in page_data:
                for region in page_data['refined_regions']:
                    if 'tags' in region:
                        for tag in region['tags']:
                            process_tag(tag)
    
    return {
        'level1': level1_tags,
        'level2': level2_tags,
        'level3': level3_tags,
        'full_tags': full_tags
    }

tag_statistics = extract_and_analyze_tags(data_dict)

# Calculate unique tag counts
unique_level1_tags = len(tag_statistics['level1'])
unique_level2_tags = len(tag_statistics['level2'])
unique_level3_tags = len(tag_statistics['level3'])
total_unique_tags = len(tag_statistics['full_tags'])

# Print statistics and unique counts
print("Level 1 Tags:", tag_statistics['level1'])
print("Unique Level 1 Tags:", unique_level1_tags)
print("Level 2 Tags:", tag_statistics['level2'])
print("Unique Level 2 Tags:", unique_level2_tags)
print("Level 3 Tags:", tag_statistics['level3'])
print("Unique Level 3 Tags:", unique_level3_tags)
print("Full Tags Count:", tag_statistics['full_tags'])
print("Total Unique Tags:", total_unique_tags)


Level 1 Tags: Counter({'aml': 43085})
Unique Level 1 Tags: 1
Level 2 Tags: Counter({'aml-cdd': 7645, 'aml-customeridentification': 5187, 'aml-definitions': 4547, 'aml-str': 3991, 'aml-pcp': 3426, 'aml-riskassessment': 2341, 'aml-penalties': 2140, 'aml-assetfreeze': 2122, 'aml-tfs': 2092, 'aml-other': 2057, 'aml-competentauthority': 2010, 'aml-recordkeeping': 1516, 'aml-mlro': 977, 'aml-wiretransfer': 869, 'aml-activitybasedsanction': 774, 'aml-intlcooperation': 755, 'aml-bo': 505, 'aml-sanctionrights': 98, 'aml-internationalcooperation': 33})
Unique Level 2 Tags: 19
Level 3 Tags: Counter({'aml-definitions-other': 2724, 'aml-customeridentification-verification': 1348, 'aml-str-firms': 1015, 'aml-penalties-other': 586, 'aml-assetfreeze-other': 500, 'aml-pcp-amlprogram': 436, 'aml-cdd-measures': 417, 'aml-cdd-enhanced': 390, 'aml-wiretransfer-information': 371, 'aml-competentauthority-other': 353, 'aml-recordkeeping-other': 331, 'aml-customeridentification-other': 314, 'aml-pcp-other': 27

### Snippet Code

In [ ]:
# TODO:

# This is where RegGenome's snippeting algorithm would need to be added!!!
# (deleted here for confidentitiality reasons)

In [11]:
# define names for character types:
character_type_descriptions = {
    0: "(e.g., article 1 or article1)",
    1: "(e.g., '1.')",
    2: "(e.g., '(1)')",
    3: "(e.g., '1)')",
    4: "(e.g., '1)' or '1 ')",
    5: "(e.g., '1.1.')",
    6: "(e.g., '1.1.1.')",
    7: "(e.g., '1-1')",
    8: "(e.g., 'letterletter' all combinations until 'ez')",
    9: "(e.g., 'article I')",
    10: "(e.g., 'article (1)')",
    11: "(e.g., 'rule 1')",
    12: "(e.g., artículo 1 or artículo1)",
    13: "(e.g., 'artículo I')",
    None: "(Split by Size)"
}

### Run an exemplary page from a file through algorithm

In [12]:
import re

def normalize_whitespace(text):
    '''
    Replaces multiple whitespace characters with a single space and trims leading/trailing spaces.
    '''
    return re.sub(r'\s+', ' ', text).strip()

In [13]:
doc_id = '17144355'
page_index = '1'
full_text = data_dict[doc_id][page_index]['full_text']

split_page_into_snippets(full_text, page_index) # returns list of snippets (which are dicts)

# the below output shows the resuts of the snippeting algorithm

([{'snippet_index': 0,
   'offset': 0,
   'limit': 2381,
   'text': 'Model 1 IGA Annex I November 30, 2014   2   individuals (“Preexisting Individual Accounts”).  A. Accounts Not Required to Be Reviewed, Identified, or Reported.  Unless the Reporting [FATCA Partner] Financial Institution elects otherwise, either with respect to all Preexisting Individual Accounts or, separately, with respect to any clearly identified group of such accounts, where the implementing rules in [FATCA Partner] provide for such an election, the following Preexisting Individual Accounts are not required to be reviewed, identified, or reported as U.S. Reportable Accounts:  1.  Subject to subparagraph E(2) of this section, a Preexisting Individual Account with a balance or value that does not exceed $50,000 as of the Determination Date.  2. Subject to subparagraph E(2) of this section, a Preexisting Individual Account that is a Cash Value Insurance Contract or an Annuity Contract with a balance or value of $250,

### Create a Comparison Table

In [15]:
def draw_boxes_on_image(img, boxes, regions_labels, color='red', width=4, font_path="~/.fonts/Times New Roman.ttf", font_size=12):
    '''
    Function to draw boxes on an image and add labels to them.
    Based on the provided boxes and labels, this function will draw rectangles around the boxes and add labels above them similar to how RegGenome annotators would have done.
    '''
    draw = ImageDraw.Draw(img)
    font_path = os.path.expanduser(font_path)
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        # Fallback to default font if specified font is not found
        font = ImageFont.load_default()
    
    font.size = font_size
    for box, labels in zip(boxes, regions_labels):
        left, top, w, h = box
        right = left + w
        bottom = top + h

        # Draw rectangle, similar to what annotators at RegGenome would have done
        draw.rectangle([left, top, right, bottom], outline=color, width=width)

        # Prepare label text
        text = f'Region Tag: {", ".join(labels)}' if labels else 'Region Tag: No Tag'
        
        # Calculate text size with font
        text_left, text_top, text_right, text_bottom = font.getbbox(text)
        text_width = text_right - text_left
        text_height = text_bottom - text_top
        
        # Calculate text position (centered above the box)
        text_x = left + (w - text_width) / 2
        text_y = top - text_height - 5

        # Draw text
        draw.text((text_x, text_y), text, fill=color, font=font)

    return img

In [24]:
import uuid

def convert_pdf_page_to_image(pdf_path, page_index, output_path, local_file=True, zoom_factor_image=6):
    '''
    Function to convert a PDF page to an image
    '''
    
    # Open the PDF file by either loading it from disk or downloading it from a URL
    if local_file:
        doc = fitz.open(pdf_path)
    else:
        request = requests.get(pdf_path)
        filestream = io.BytesIO(request.content)
        doc = fitz.open(stream=filestream, filetype="pdf")

    page = doc.load_page(page_index)

    # Apply zoom factor for better quality
    mat = fitz.Matrix(zoom_factor_image, zoom_factor_image)
    pix = page.get_pixmap(matrix=mat)

    pix.save(output_path)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    return output_path, img

def convert_pdf_page_to_image_with_regions(pdf_path, page_index, regions_content, output_path, local=True, zoom_factor=6):
    '''
    Function to convert a PDF page to an image, draw regions on it, and save it to a file
    '''
    output_path, img = convert_pdf_page_to_image(pdf_path, page_index, output_path, local_file=local, zoom_factor_image=zoom_factor)
    regions_info = [(region['text'], float(region['x']), float(region['y']), float(region['width']), float(region['height'])) for region in regions_content]

    # Scale and draw regions with labels
    img_width, img_height = img.size
    region_boxes = [(x * img_width, y * img_height, width * img_width, height * img_height) for _, x, y, width, height in regions_info]
    region_labels = [region['tags'] if region['tags'] else 'No Tag' for region in regions_content]
    font_path = "~/.fonts/Times New Roman.ttf" if is_VM else "/c/Windows/Fonts/times.ttf"
    img_with_boxes = draw_boxes_on_image(img, region_boxes, region_labels, color='red', width=2*zoom_factor, font_path=font_path, font_size=12*zoom_factor)

    # Save updated image
    img_with_boxes.save(output_path)
    return output_path

def highlight_text(full_text, segments, style_odd, style_even):
    '''
    Highlights segments of text in the full text with alternating styles
    '''

    full_text_normalized = normalize_whitespace(full_text)
    highlighted_segments = []
    for i, segment in enumerate(segments):
        segment_normalized = normalize_whitespace(segment)
        
        # Determine style and highlight segment
        style = style_odd if i % 2 == 0 else style_even
        highlighted_segment = f'<span class="{style}">{segment_normalized}</span>'

        # Replace normalized segment in full text with highlighted version
        full_text_normalized = full_text_normalized.replace(segment_normalized, highlighted_segment, 1)

        highlighted_segments.append(highlighted_segment)
    
    # Return full text with highlighted segments
    return full_text_normalized, highlighted_segments

def create_comparison_table(doc_id, page_index, data_dict, pdf_image_path=None):
    '''
    Creates a custom HTML table for comparing full page text, blocks, snippets, and optionally regions, with an optional image of the original page.
    '''

    styles = open('style.css', "r").read() # Load CSS styles for table

    # Get elements from dict
    document_title = data_dict[doc_id].get('title', '')
    full_page_text = data_dict[doc_id][page_index]['full_text']
    blocks = [block['text'] for block in data_dict[doc_id][page_index]['blocks'] if block['text']]

    # Get snippets
    snippets_text = []
    snippets = data_dict[doc_id][page_index]['snippets']
    snippet_character_type = data_dict[doc_id][page_index]['snippets'][0]['snippet_character_type']
    for snippet in snippets:
        snippets_text.append(snippet['text'])

    # Check if regions exist in the data_dict
    regions_exist = 'regions' in data_dict[doc_id][page_index]
    regions = []
    if regions_exist:
        for region in data_dict[doc_id][page_index]['regions']:
                region_tags = region.get('tags', [])
                region_tags_formatted = '<br>'.join(region_tags)
                region_text = region['text']
                regions.append(f'<strong>{region_tags_formatted}</strong><br>{region_text}')

    # Same for adjusted regions
    refined_regions_exist = 'refined_regions' in data_dict[doc_id][page_index]
    refined_regions = []
    if refined_regions_exist:
        for refined_region in data_dict[doc_id][page_index]['refined_regions']:
            refined_region_tags = refined_region.get('tags', [])
            refined_region_tags_formatted = '<br>'.join(refined_region_tags)
            refined_region_text = refined_region['text']
            refined_regions.append(f'<strong>{refined_region_tags_formatted}</strong><br>{refined_region_text}')

    # Define styles for blocks and snippets
    block_style_odd = "block-odd"
    block_style_even = "block-even"
    snippet_style_odd = "snippet-odd"
    snippet_style_even = "snippet-even"

    # Highlight text segments
    full_text_highlighted, highlighted_snippets = highlight_text(full_page_text, snippets_text, snippet_style_odd, snippet_style_even)
    full_text_highlighted, highlighted_blocks = highlight_text(full_text_highlighted, blocks, block_style_odd, block_style_even)

    # Determine maximum number of rows for the table
    max_rows = max(len(blocks), len(regions), len(snippets))

    # Building HTML table
    html_table = '<table class="comparison-table">'
    columns_count = 3
    if regions_exist:
        columns_count += 1
    if refined_regions_exist:
        columns_count += 1
    html_table += f'<tr class="table-title"><th colspan="{columns_count}">{document_title} (Page {int(page_index)})</th></tr>'

    # HEADERS
    table_header = '<tr class="table-header">'
    table_header += '<th>Full Text of Page</th>'
    table_header += '<th>Blocks</th>'
    table_header += f'<th>Snippets<br><span style="font-size: 12pt; font-style: italic;">(Char. Type: {snippet_character_type} {character_type_descriptions.get(snippet_character_type, "Uknown")})</span></th>'
    if regions_exist:
        table_header += '<th>Regions</th>'
    if refined_regions_exist:
        table_header += '<th>Refined Regions</th>'
    html_table += table_header + '</tr>'

    # ADDING ROWS
    for i in range(max_rows):
        html_table += '<tr>'
        
        if i == 0:
            html_table += f'<td rowspan="{max_rows}" class="full-text">{full_text_highlighted}</td>'

        block_text = highlighted_blocks[i] if i < len(highlighted_blocks) else ''
        html_table += f'<td style="width: 25%;">{block_text}</td>'
        
        snippet_row_span = max_rows // len(snippets_text) if snippets_text else 1
        region_row_span = max_rows // len(regions) if regions and regions_exist else 1

        if i % snippet_row_span == 0 and i // snippet_row_span < len(snippets_text):
            html_table += f'<td rowspan="{snippet_row_span}" class="snippets">{highlighted_snippets[i // snippet_row_span]}</td>'
        elif i == 0:
            html_table += '<td></td>'

        if regions_exist:
            if i % region_row_span == 0 and i // region_row_span < len(regions):
                html_table += f'<td rowspan="{region_row_span}" class="regions">{regions[i // region_row_span]}</td>'
            elif i == 0:
                html_table += '<td></td>'
        if refined_regions_exist:
            if i % region_row_span == 0 and i // region_row_span < len(regions):
                html_table += f'<td rowspan="{region_row_span}" class="regions">{refined_regions[i // region_row_span]}</td>'
            elif i == 0:
                html_table += '<td></td>'
        html_table += '</tr>'

    html_table += '</table>'

    # Create container for image and table
    html_style = f'<style>{styles}</style>'
    html_content = '<div style="display: flex;">'
    if pdf_image_path:
        html_content += f'<img src="{pdf_image_path}">'
    html_content += html_style + html_table
    html_content += '</div>'

    with open("comparison_table.html", "w", encoding="utf-8") as file:
        file.write(html_content)

    return html_content

def create_comparison(dict_to_use, doc_id, page_index, pdf_path, is_pdf_local=False):
    '''
    Main function to create a comparison table for a given document ID and page index
    '''
    output_path = f"../Example_PDFs/current_page_image_{random()}.png"

    # Convert PDF page to an image
    if is_pdf_local:
        convert_pdf_page_to_image(pdf_path, page_index, output_path, is_pdf_local)
    else:
        convert_pdf_page_to_image_with_regions(pdf_path, page_index, dict_to_use[doc_id][str(page_index)]["regions"], output_path, is_pdf_local)

    # Generate the table HTML
    table_html = create_comparison_table(doc_id, str(page_index), dict_to_use, output_path)

    # Display the HTML
    display(HTML(table_html))  # Use this in a Jupyter notebook or similar environment
    time.sleep(2)
    os.remove(output_path) # remove temp png
    
def get_regvision_link(document_id: str):
    '''
    Function to generate a link to the pdf file via a RegVision link with RegGenome's fingerprinting
    '''
    ########## REMOVED FOR CONFIDENTIALITY REASONS ##########

def create_comparison_table_interactive(data_dict):
    '''
    Function to create an interactive widget (i.e. the comparison tool) for selecting a document ID and page index to compare the full text, blocks, and snippets.
    '''
    def update_analysis(doc_id, page_index):
        
        if PROJECT == "AML":
            pdf_path = ########## REMOVED FOR CONFIDENTIALITY REASONS ##########
        elif PROJECT == "CYBER_I":
            pdf_path = get_regvision_link(str(data_dict[doc_id]['doc_long_id']))
        elif PROJECT == "CYBER_II":
            pdf_path = get_regvision_link(str(data_dict[doc_id]['doc_long_id']))
        else:
            raise "Wrong Project."
        create_comparison(data_dict, doc_id, page_index, pdf_path, is_pdf_local=False)

    # Dropdown for document IDs
    doc_id_dropdown = widgets.Dropdown(options=sorted(list(data_dict.keys())), description='Document ID:')

    # Slider for page index
    page_slider = widgets.IntSlider(min=0, max=1, step=1, description='Page:', continuous_update=False)

    def update_page_range(*args):
        doc_id = doc_id_dropdown.value
        max_page = len(data_dict[doc_id])-1 if doc_id in data_dict else 1
        page_slider.max = max_page

    # Update page slider range when new document is selected
    doc_id_dropdown.observe(update_page_range, 'value')

    interactive_widget = widgets.interactive(update_analysis, doc_id=doc_id_dropdown, page_index=page_slider)
    display(interactive_widget)
    
######################################### 
create_comparison_table_interactive(data_dict)

interactive(children=(Dropdown(description='Document ID:', options=('17144355', '17144946', '17145117', '17146…

# Custom PDF Example
Create comparison tool for a custom pdf

In [19]:
# Function to create a default page structure
def default_page():
    return {'blocks': [], 'snippets': [], 'full_text': ""}

def create_adjusted_dict(pdf_path: str, doc_id: str):
    pdf_data_dict = defaultdict(lambda: defaultdict(default_page))

    doc = fitz.open(pdf_path)
    # Process each page in the PDF
    for page_index, page in enumerate(doc):
        # Extract text blocks from the page
        blocks = page.get_text("blocks")

        # Process each block
        block_texts = []
        for block in blocks:
            block_text = block[4].strip()  # Extract text from the block
            block_texts.append(block_text)
            pdf_data_dict[doc_id][str(page_index)]['blocks'].append({'text': block_text})
        full_page_text = ' '.join([block_text for block_text in block_texts if block_text is not None])
        pdf_data_dict[doc_id][str(page_index)]['full_text'] = full_page_text
        snippets, snippet_character_type = ##### here we would call the snippeting algorithm #####
        for snippet in snippets:
            pdf_data_dict[doc_id][str(page_index)]['snippets'].append({'text': snippet['text'], 'snippet_character_type': snippet_character_type})
    return pdf_data_dict


####################################################################

page_index = 1
file_index = 0

pdf_files = [file for file in os.listdir("../Example_PDFs") if file.endswith('.pdf')]
pdf_path = os.path.join("../Example_PDFs", pdf_files[file_index]) 

# create dict for local file
adjusted_dict = create_adjusted_dict(pdf_path, 'test_pdf')

print(f"File: {pdf_path}")
create_comparison(adjusted_dict, 'test_pdf', page_index, pdf_path, is_pdf_local=True)


File: ../Example_PDFs/EXAMPLE1_finregulation.pdf
